# Pricing and Yields of United States Treasury Notes and Bounds

## Background
Treasury notes, also called T-notes and treasury bonds, provide a fixed interest rate throughout their lifespan and pay semi-annual coupon payments until they reach maturity. When the note (or bond) matures, the holder receives the face (or par) value of the note (or bond). This sets them apart from treasury bills, as treasury notes (and bonds) offer coupon payments every six months, along with earning interest, for the duration of the security. Treasury notes are offered for 2, 3, 5, 7, and 10 years, whereas bonds mature after 20 or 30 years. They can be priced below or above their face (or par) value, but the lenders are entitled to the full par value of the note upon maturity.

<div>
    <center>
        <img src="figs/Fig-Bond-Asset-Timeline-Schematic.png" width="640"/>
    </center>
</div>


### Pricing
Let the term of a note be T-years with $\lambda$ coupon payments per year; $N = \lambda{T}$ coupon payments over the bond term. Further, let $\bar{c}$ denote the annualized coupon rate, and $\bar{r}$ denote the annual effective market rate of interest. Then, the _fair price_ for note $V_{B}$ is the present value of coupon payments $C$ plus the discounted face (par) value $V_{P}$ of the bond:

$$
V_{B} = \mathcal{D}^{-1}_{N}V_{P}+\sum_{j=1}^{N}\mathcal{D}_{j}^{-1}C
$$

where $\mathcal{D}_{i}$ denotes the discount factor for the period $0\rightarrow{I}$, which can be either a discrete or continuous compounding model. The coupon payment $C=\left(\bar{c}/\lambda\right)\cdot{V_{P}}$ is set when the bond is purchased, and the interest rate $i=\bar{r}/\lambda$ varies with the market (set at auction). The contract between the U.S. government, the issuer, and the note holder (you) includes the note’s coupon rate, maturity date, and face (par) value.

## Learning objectives
In this example, we will explore the pricing and yield of coupon-paying treasury notes and bonds.

* __Objective 1__: First, we'll validate the prices calculated by implementing the pricing formula above using historical data for bonds and notes of different durations. 
* __Objective 2__: Next, using our validated bond/note code, compute the [Yield to Maturity (YTM)](https://en.wikipedia.org/wiki/Yield_to_maturity) for bonds or notes of different durations. In particular, we'll show how the bond (or note) parameters influence the yield to maturity. 

## Setup
In the following code blocks, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this exercise, and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).
* [Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/) is a [Julia](https://julialang.org) package for performing simple optimization task. We use this package in the Yield to Maturity (YTM) calculations.
* [PrettyTables.jl](https://ronisbr.github.io/PrettyTables.jl/stable/man/html_backend/) is a [Julia](https://julialang.org) package that implements functions to construct and display text-based tables

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryCouponSecurityModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$, the market rate of interest rate $\bar{r}$, the coupoun rate $c$ and the number of coupon payments per year $\lambda$ for treasury note and bonds that pay a coupoun. You construct a `MyUSTreasuryCouponSecurityModel` instance using the `build` method described below.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/):

`loaddatafile(path::String) -> DataFrame` 
 > This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) path arguement that points to a local comma seprated value file that holds bond rate and pricing information. The pricing and rate information is returned to the caller in a [DataFrame](https://dataframes.juliadata.org/stable/).
 
`build(model::Type{MyUSTreasuryCouponSecurityModel}, data::NamedTuple) -> MyUSTreasuryCouponSecurityModel` 
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, etc.) and returns an instance of the `MyUSTreasuryCouponSecurityModel` [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types).

`price(model::MyUSTreasuryCouponSecurityModel, compounding::T) -> MyUSTreasuryCouponSecurityModel where T <: AbstractCompoundingModel` 
> This function takes a `MyUSTreasuryCouponSecurityModel` instance, and a compounding model and returns an updated `MyUSTreasuryCouponSecurityModel` instance that holds the price $V_{B}$ in the price field.

`YTM(model::MyUSTreasuryCouponSecurityModel, compounding::T; rₒ::Float64 = 0.01) where T <: AbstractCompoundingModel` 
> This function computes the Yield to Maturity (YTM) given a `MyUSTreasuryCouponSecurityModel` instance, a compounding model and an initial guess for the interest rate (default value of 1\%)

In [1]:
include("Include.jl");

## Objective 1: Validate the Coupon Bond Pricing Implementation
We have conducted three examples of coupon note (bond) pricing to verify our treasury note (bond) implementation and gain a better understanding of how note (bond) market factors influence prices:

- Example 1: We have calculated the price of a 5-year treasury note with semi-annual coupon payments using continuous compounding.
- Example 2: We have also computed the price of a 2-year treasury note with annual coupon payments using discrete compounding.
- Example 3: Lastly, we have compared the 30-year treasury bond prices we calculated with historical data from auctions held between 1977 and 1979, assuming discrete compounding.

To calculate the price of a note or a bond, we create an instance of the `MyUSTreasuryCouponSecurityModel` type using the `build(...)` function and then pass this instance to the `price` function. The function will then update the model with a calculated price, populating the `price` field of the `MyUSTreasuryCouponSecurityModel` instance.

### Example 1: Compute the price of a 5-year treasury note with semi-annual coupon payments
Compute the price of a 5-year treasury note with semi-annual coupons with a coupon rate of 6% and a market rate of interest $\bar{r}$ of 6.76%. The face value of the note is 100 USD. We expect a price of the T-Note to be 98.39 USD. This example was reproduced from Chapter 4 of [Hull, Options, Futures and other Derivatives, 7th ed, Pearson Education, 2009](). This example uses continuous compounding.

Let's set the values for the 5-year treasury note (par value $V_{P}$, duration in years $T$, market rate of interest $\bar{r}$, coupon rate $c$ and the number of coupon payments per year $\lambda$), the known price (`98.39` USD), the compounding model (an instance of the `ContinuousCompoundingModel` type which we store in the `continuous_compounding` variable):

In [2]:
Vₚ,T,r̄,c,λ = 100.0,2.0,0.0676,0.06,2
expected_answer = 98.39;
continuous_compounding = ContinuousCompoundingModel();

We can then make use of the [Julia piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) `|>` and some other syntactic sugar to build the treasury note (bond) model instance (an instance of the `MyUSTreasuryCouponSecurityModel` type stored in the `model` variable) , and compute the price in a single block of code, with either continuous or discrete compounding:

In [3]:
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r̄, coupon = c, λ = λ
        )) |> continuous_compounding;

To display the result, we use the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. In particular, we store the expected the computed prices in the `example_1_data_array`, along with the percentage difference between these values. The data is displayed as a table using a call to the `pretty_table(...)` function: 

In [4]:
example_1_data_array = Array{Any,2}(undef, 3,2);
example_1_header_data = (["","Value"]);
example_1_data_array[1,1] = "Reported (USD)"
example_1_data_array[1,2] = expected_answer
example_1_data_array[2,1] = "Calculated (USD)"
example_1_data_array[2,2] = round(model.price, sigdigits=4)
example_1_data_array[3,1] = "Error (%)"
example_1_data_array[3,2] = ((expected_answer - model.price)/(expected_answer))*100

pretty_table(example_1_data_array, header=example_1_header_data, tf = tf_html_default);

,Value
Reported (USD),98.39
Calculated (USD),98.39
Error (%),0.000351637


### Example 2: Compute the price of a 2-year treasury note with annual coupon payments
Compute the price of a 2-year treasury note with annual coupons with a coupon rate of 7% and a market rate of interest of 8%, assuming discrete compounding. The face value of the note is $V_{P}=$100 USD. This example was reproduced from [FINA 635 course, Department of Finance at the C.T. Bauer College of Business, University of Houston.](https://www.bauer.uh.edu/nlangberg/teaching_files/Fall2010/class%205_2008sum.pdf) The expected is $V_{B}$ = 98.22 USD. Example 2 is similar to the previous example, except the T-Note has a single coupon payment per year ($\lambda = 1$), and we use discrete compounding:

In [5]:
Vₚ,T,r̄,c,λ = 100.0,2.0,0.08,0.07,1
discrete_compounding = DiscreteCompoundingModel();
expected_answer = 98.22;

We use the [Julia piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) `|>` to build the treasury note (bond) model instance (an instance of the `MyUSTreasuryCouponSecurityModel` type stored in the `model` variable) , and compute the price in a single block of code, with either continuous or discrete compounding:

In [6]:
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r̄, coupon = c, λ = λ
        )) |> discrete_compounding;

To display the result, we use the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. In particular, we store the expected the computed prices in the `example_2_data_array`, along with the percentage difference between these values. The data is displayed as a table using a call to the `pretty_table(...)` function: 

In [7]:
example_2_data_array = Array{Any,2}(undef, 3,2);
example_2_header_data = (["","Value"]);
example_2_data_array[1,1] = "Reported (USD)"
example_2_data_array[1,2] = "98.22"
example_2_data_array[2,1] = "Calculated (USD)"
example_2_data_array[2,2] = round(model.price, sigdigits=4)
example_2_data_array[3,1] = "Error (%)"
example_2_data_array[3,2] = ((expected_answer - model.price)/(expected_answer))*100

pretty_table(example_2_data_array, header=example_2_header_data, tf = tf_html_default);

,Value
Reported (USD),98.22
Calculated (USD),98.22
Error (%),0.00332391


### Example 3: Comparison of calculated 30-year treasury bond prices with historical data
Lastly, we compare the 30-year treasury bond prices we calculated with historical data from auctions held between 1977 and 1979, assuming discrete compounding. The data for this example was reproduced from the [Treasury Notes & Bonds Historical Information page on Treasury.gov](https://treasurydirect.gov/auctions/announcements-data-results/announcement-results-press-releases/previous-announcements-and-results/treasury-notes-bonds-historical-information/). The data consists of 30-year treasury bond information (coupon rate, market interest rate, and price) for bonds issued between 1977 and 1979.

#### Strategy
1. First, let's load the historical data set using the `loaddatafile` function, which returns the bond data in as a [DataFrame](https://dataframes.juliadata.org/stable/) with the key fields:
    * The `:CouponRate` field holds the data for the coupon interest rate (annualized rate as a percentage)
    * The `:AverageYield` field holds the market rate of interest for the treasury bond (annualized rate as a percentage)
    * The `:AveragePrice` field holds the price data for the 30-year treasury bond
1. Next, we'll compute the treasury bond price for each bond in the data set by iterating through the data set using a `for` loop using the shortcut syntax shown above in the examples
1. Last, we'll store the actual and computed prices in a table and calculate the percentage error of the estimate.

#### Load the historical data set
The data is stored in a local comma-separated value file named `Data-30-yr-USTreasury-1977-1979.csv`. We create the path to the data file and store this value in the `path_to_data_file` variable. We then pass the path to the `loaddatafile(...)` function. The bond data is stored as a [DataFrame](https://dataframes.juliadata.org/stable/) in the `dataset` variable:

In [8]:
path_to_data_file = joinpath(_PATH_TO_DATA, "Data-30-yr-USTreasury-1977-1979.csv");
dataset = loaddatafile(path_to_data_file);

#### Process each record (row) in the historical data set using a `for` loop
We first set the invariant parameters, e.g., the maturity $T$ in years, the number of coupon payments per year $\lambda$, and the face value of the bond $V_{P}$. We then determine how many records we have in the `dataset` using the `nrow(..)` function. Finally, we create the `bonds` array, which holds instances of `MyUSTreasuryCouponSecurityModel` for each bond in the data set, and process each record in a `for` loop:

In [9]:
T,λ,Vₚ = 30.0, 2, 100.0
number_of_records = nrow(dataset);
bonds = Array{MyUSTreasuryCouponSecurityModel,1}(undef,number_of_records);

for i ∈ 1:number_of_records
    
    # get the data from the data frame for this record (data is in percentage, convert to decimal)
    c = dataset[i,:CouponRate]*(1/100);
    r̄ = dataset[i,:AverageYield]*(1/100);
    
    # build the model, and compute the price using the short-cut syntax
    model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r̄, coupon = c, λ = λ
        )) |> discrete_compounding;
    
    # store the updated model
    bonds[i] = model;
end

#### Build a comparison table of actual versus computed 30-yr bond prices
To display the calculated versus historical bond prices, we use the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. In particular, we store the historical $V_{B}$ the calculated $\hat{V}_{B}$ prices, along with the other bond parameters and percentage difference between the calculated and historical values $\epsilon$ in the `example_3_data_array`. The array is displayed as a table using a call to the `pretty_table(...)` function: 

In [10]:
example_3_data_array = Array{Any,2}(undef, number_of_records, 7);
example_3_header_array = (
    ["Date","Duration","Coupon","Yield", "Vᵦ (actual)", "Vᵦ (computed)", "Error ϵ"], # descriptions
    ["MM/DD/YY","YR","%","%","USD","USD","%"] # units
);

for i ∈ 1:number_of_records
        
    # get data from data set, and model array -
    date = dataset[i,:AuctionDate];
    c = dataset[i,:CouponRate]
    r = dataset[i,:AverageYield]
    Vᵦ = dataset[i,:AveragePrice];
    V̂ᵦ = bonds[i].price
    
    ϵ = ((Vᵦ - V̂ᵦ)/Vᵦ)*100.0
    
    # put data into the table array -
    example_3_data_array[i,1] = date
    example_3_data_array[i,2] = T
    example_3_data_array[i,3] = c
    example_3_data_array[i,4] = r
    example_3_data_array[i,5] = Vᵦ
    example_3_data_array[i,6] = V̂ᵦ
    example_3_data_array[i,7] = ϵ
end

pretty_table(example_3_data_array, header=example_3_header_array, tf = tf_html_default)

Date,Duration,Coupon,Yield,Vᵦ (actual),Vᵦ (computed),Error ϵ
MM/DD/YY,YR,%,%,USD,USD,%
5/8/75,30.0,8.25,8.3,99.45,99.4501,-0.000112277
2/4/77,30.0,7.625,7.63,99.941,99.9414,-0.000401125
11/2/77,30.0,7.875,7.94,99.261,99.2605,0.000464192
8/3/78,30.0,8.375,8.43,99.402,99.4024,-0.000362882
11/3/78,30.0,8.75,8.86,98.851,98.8506,0.000400852
5/2/79,30.0,9.125,9.23,98.938,98.9383,-0.000327602
11/1/79,30.0,10.375,10.44,99.407,99.4068,0.00020879


### Summary of Objective 1
We have successfully implemented a pricing model for coupon-paying treasury securities (treasury notes and bonds) that is based on historical data and real-world examples. This gives us the confidence to accurately calculate the price of a coupon note or bond using market information such as maturity, coupon rate, and interest rates.

## Objective 2: Calculating the Yield to Maturity (YTM) for Treasury Notes and Bonds

The Yield to Maturity (YTM) of a treasury note or bond is the [internal rate of return (IRR)](https://en.wikipedia.org/wiki/Internal_rate_of_return) associated with buying and holding the security until its maturity date. Thus, initially, the YTM and the effective market interest rate are the same when the note or bond is auctioned in the primary market. However, treasury securities can be resold on the [secondary treasury market](https://www.investopedia.com/articles/bonds/08/treasuries-fed.asp#:~:text=For%20many%20people%2C%20TreasuryDirect%20is,convenience%20and%20liquidity%20than%20TreasuryDirect), even after some coupon payments have been paid out. In such cases, the YTM is not equal to the initial market interest rate at auction. Let's examine the yield to maturity of a 5-year treasury note.

Assuming a discrete discounting model, let's consider a scenario where we bought a 5-year treasury note with a face value of $V_{P}$ = 100 USD at auction for $V_{B}$ = 96.82 USD. The note has an annual coupon rate of 6%, with semi-annual coupons. We then sell the T-note on the secondary market for the same price we purchased it, after holding it for $T^{\prime}$ years and receiving some coupon payments.

### Compute YTM at the primary auction:
Set the values for the 5-year treasury note (par value $V_{P}$, price $V_{B}$, duration in years $T$, coupon rate $c$ and the number of coupon payments per year $\lambda$), and the compounding model, an instance of the `DiscreteCompoundingModel` type which we store in the `compounding` variable:

In [11]:
Vₚ, Vᵦ, T, c, λ = 100.0, 96.82, 5.0, 0.06, 2
compounding = DiscreteCompoundingModel();

Next, build an instance of the `MyUSTreasuryCouponSecurityModel` type using the `build(...)` function (store this in the `model` variable), set the price on the `model` (which is known in this case), and then compute the yeild to maturity using the `YTM(...)` function. Finally, display the yield to maturity using the `println(...)` function:

In [23]:
# build the Bond model, we don't know the rate, but we know the price -
model = build(MyUSTreasuryCouponSecurityModel, (
    par = Vₚ, T = T, λ = λ, coupon = c
));
model.price = Vᵦ

# compute the rate -
estimated_rate = YTM(model, compounding)

# print -
println("A 5-year note with 100 USD face value purchased at auction for $(Vᵦ) USD has a YTM = $(estimated_rate*100) percent")

A 5-year note with 100 USD face value purchased at auction for 96.82 USD has a YTM = 6.760119063927231 percent


### Compute the YTM after holding for $T^{\prime}$ years and resale on the secondary market
If we hold the note for some time (at least six months), we'll receive coupon payments. Upon resale in the secondary market, these coupon payments will not be available to the new owner of the note. However, the face value of the note has stayed the same. Additionally, the time to maturity (the number of periods of the note) will be reduced. Thus, the YTM on the note (bond) will differ from the note’s initial market interest rate.

Let's compute the YTM after waiting $T^{\prime}$ years to resell the note. First, initialize an array to hold the YTM values, and set the values for the 5-year treasury note (par value $V_{P}$, price $V_{B}$, duration in years $T$, coupon rate $c$ and the number of coupon payments per year $\lambda$), and the compounding model, an instance of the `DiscreteCompoundingModel` type which we store in the `compounding` variable. We assume we resell the note for the same price that we initially purchased it for:

In [14]:
Vₚ, Vᵦ, T, c, λ = 100.0, 96.82, 5.0, 0.06, 2
YTM_value_array = Array{Float64,1}(undef,5);
compounding = DiscreteCompoundingModel();

Next, we set up a holding time array, stored in the variable $\Delta{T}$, which contains the number of years that we hold the note. We then process each element in the $\Delta{T}$ array. During each pass through the loop, we compute the remaining time on the note (store this value in the $\hat{T}$ variable), set the price, and then compute the yield to maturity by calling the `YTM(...)` function:

In [15]:
ΔT = [0.0,1.0,2.0,3.0,4.0];
counter = 0;
for T′ ∈ ΔT 
    
    # update the counter -
    counter += 1
    
    # how much time is left on the security?
    T̂ = T - T′
    
    # build a bond model, we don't know the rate, but we know the price. 
    model = build(MyUSTreasuryCouponSecurityModel, (
        par = Vₚ, T = T̂, λ = λ, coupon = c
    ));
    model.price = Vᵦ
    
    # We compute the yeild and store in the YTM_value_array
    YTM_value_array[counter] = YTM(model, compounding)
end

### Display the YTM values
To display the YTM values, we use the [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package. In particular, we store the $\Delta{T}$ values, the remaining time to maturity for the note, $\hat{T}$, the price (assumed to be constant), the initial effective market rate of interest $\bar{r}$ at auction, and the YTM values for each $\Delta{T}$ value in the `YTM_table_data_array` array. The array is displayed as a table by calling the `pretty_table(...)` function: 

In [32]:
YTM_table_data_array = Array{Any,2}(undef, counter, 5)
YTM_table_header_array = (["Elapsed time ΔT","Remaining time T̂","Price","Effective r̄", "YTM"], ["YR","YR","USD","%", "%"]);

rowindex = 0;
for T′ ∈ ΔT 
    
    # update the counter -
    rowindex += 1
    
    # how much time is left on the security?
    T̂ = T - T′

    # populate the table data array -
    YTM_table_data_array[rowindex,1] = T′
    YTM_table_data_array[rowindex,2] = T - T′
    YTM_table_data_array[rowindex,3] = Vᵦ;
    YTM_table_data_array[rowindex,4] = estimated_rate*100
    YTM_table_data_array[rowindex,5] = YTM_value_array[rowindex]*100
end

pretty_table(YTM_table_data_array, header = YTM_table_header_array, tf = tf_html_default)

Elapsed time ΔT,Remaining time T̂,Price,Effective r̄,YTM
YR,YR,USD,%,%
0.0,5.0,96.82,6.76012,6.76012
1.0,4.0,96.82,6.76012,6.92376
2.0,3.0,96.82,6.76012,7.19744
3.0,2.0,96.82,6.76012,7.74689
4.0,1.0,96.82,6.76012,9.40605


### Summary of Objective 2
The example demonstrated that when the time to maturity decreases while the price remains fixed, the yield to maturity for a treasury note increases. This is logical since the note always pays its face (par) value at maturity. Therefore, if the new owner acquires the note at the same price as the initial buyer, they will receive the face value (as well as the remaining coupon payments) over a shorter period.